## Blink Features
The notebook should show how each of the features is calculated from the raw eye closure signal.
1. Test data is created
2. Blink events are identified
3. Blink events are validated visually
4. Start and Stop of each event is adjusted
5. Different Variables are calculated from eye closure signal and blink events.

In [238]:
import numpy as np
from drowsiness_detection.data import create_eye_closure_karolinksa_dataset
from drowsiness_detection.visualize import show_frame_slider
import matplotlib.pyplot as plt
import matplotlib
from sortedcontainers import SortedSet
import copy
import pandas as pd
from typing import List, Set


np.random.seed(42)

In [239]:
matplotlib.use("TkAgg") ## needed to spawn an external interactive plot

### 1. Create Test Data

In [240]:
n_samples = -1
data = next(create_eye_closure_karolinksa_dataset())
data = data["eye_closure"].to_numpy()[:n_samples]
print(data.shape, data[:5])
data_mins = len(data)//1800
print(f"data for {data_mins} mins.")
data_copy = data.copy()

Extracting file /home/tim/IM/data/potsdam_aeye_112020/001_1_a.json and response file: /home/tim/IM/data/sleep_alc_labels/001_1_a_karolinska.csv.
(168846,) [0.26446234 0.26638446 0.27353404 0.27267501 0.26480057]
data for 93 mins.


### 2. Identify Blink Events

Check for NaNs:

In [241]:
nan_mask = np.isnan(data)
print(sum(nan_mask))

18913


In [242]:
std = np.nanstd(data)
mean = np.nanmean(data)
print(f"{std=}, {mean=}")

std=0.09638660623372379, mean=0.29620456096475956


In [243]:
slider, ax = show_frame_slider(data=data)
ax.axhline(mean)
ax.axhline(mean+2*std)
ax.axhline(mean+1*std)
ax.axhline(mean+0.5*std)
plt.show(block=True)

### Validate Blink events manually
By manually looking at some blinks in the data along with different standard deviations it seems reasonable to mark every value which is 1 std. above the mean of the signal as interesting. As next step we try to find the maximum value for each group of interesting indices.

In [244]:
def blink_set_statistics(blink_sets: List[Set]):
    num_blink_events = len(blink_sets)
    mean_len = np.mean([len(s) for s in blink_sets])
    print(f"There are {num_blink_events} blink events with a mean length of {round(mean_len,2)}.")

In [245]:
def filter_points_above_threshold(threshold: float, data: np.ndarray) -> list:
    """
    """
    above_std_sets = []
    new_set = SortedSet()
    it = np.nditer(data, flags=["f_index"])
    for value in it:
        if value > threshold:
            new_set.add(it.index)
        else:
            if new_set:
                above_std_sets.append(new_set)
                new_set = SortedSet()
    return above_std_sets

MIN_CLOSURE_FOR_BLINK = mean + 2 * std
blink_sets = filter_points_above_threshold(threshold=MIN_CLOSURE_FOR_BLINK, data=data_copy.copy())

blink_set_statistics(blink_sets)
print(f"Found an average of {len(blink_sets) // data_mins} blinks per minute.")

There are 888 blink events with a mean length of 6.21.
Found an average of 9 blinks per minute.


In [246]:
def filter_blink_sets_by_length(min_len: int, sets: list):
    return [s for s in sets if len(s) > min_len]
MIN_BLINK_LENGTH = 2
blink_sets = filter_blink_sets_by_length(min_len=MIN_BLINK_LENGTH, sets=blink_sets)
print(f"After removing blinks shorter than {MIN_BLINK_LENGTH} frames there are still a total of {len(blink_sets)} blinks.")
blink_set_statistics(blink_sets)

After removing blinks shorter than 2 frames there are still a total of 691 blinks.
There are 691 blink events with a mean length of 7.58.


Visualize the center of each blink event:

In [247]:
blink_sets_means = [np.mean(list(s)) for s in blink_sets]
matplotlib.rcParams["figure.figsize"] = 5,10
fig, ax = plt.subplots()
ax.set_ylim(0,2)
ax.axhline(1, lw=5)
for mean in blink_sets_means:
    ax.axvline(mean, c="r", lw=.5)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.show(block=True)

### Adjust Start and Stop of each Blink event
Because the start or stop of a blink event can be below the threshold used for finding blinks, each blink event needs to be extended such that all eye closure changes different from the resting state are included.

In [248]:
extended_blink_sets = copy.deepcopy(blink_sets)

MIN_CHANGE_OF_BLINK = .01
for index_set in extended_blink_sets:
    start, stop = index_set[0], index_set[-1]
    while (data[start] - data[start-1]) > MIN_CHANGE_OF_BLINK:
        index_set.add(start-1)
        start -= 1
    while (data[stop] - data[stop+1]) > MIN_CHANGE_OF_BLINK:
        index_set.add(stop+1)
        stop +=1

mean_len = np.mean([len(s) for s in blink_sets])
new_mean_len = np.mean([len(s) for s in extended_blink_sets])
print(f"After extending the blink event intervals the mean length changed from {round(mean_len, 2)} to {round(new_mean_len, 2)}.")
blink_set_statistics(extended_blink_sets)

After extending the blink event intervals the mean length changed from 7.58 to 12.8.
There are 691 blink events with a mean length of 12.8.


In [249]:
# # show new boundaries with horizontal lines
# slider, ax = show_frame_slider(data=data)
# for index_set in above_std_sets_extended:
#     start, stop = index_set[0], index_set[-1]
#     ax.axvline(start, c="orange")
#     ax.axvline(stop, c="orange")
# for index_set in above_std_sets:
#     index_list = list(index_set)
#     start, stop = index_set[0], index_set[-1]
#     ax.axvline(start, c="g")
#     ax.axvline(stop, c="g")
#
#
# plt.show(block=True)

### Plot old and extended boundaries of each blink event

In [250]:
# show new boundaries with green crosses
def plot_blink_sets_start_stop(*args, blink_sets: List[Set], data: np.ndarray, slider = None, ax= None, **kwargs):
    if not slider and not ax:
        slider, ax = show_frame_slider(data=data)
    boundaries = []
    for index_set in blink_sets:
        start, stop = index_set[0], index_set[-1]
        boundaries.append(start)
        boundaries.append(stop)
    values = [data[index] for index in boundaries]
    ax.plot(boundaries, values, *args, **kwargs)
    return slider, ax

slider, ax = plot_blink_sets_start_stop("bo", blink_sets=blink_sets, data=data, ms=10)

plot_blink_sets_start_stop("g+", blink_sets=extended_blink_sets, data=data, slider=slider, ax=ax, mew=3, ms=20)
plt.show(block=True)

In [251]:
blink_sets = extended_blink_sets

### Filter out blink events with an low amplitude height

In [252]:
def filter_blink_sets_by_amplitude(data: np.ndarray, blink_sets: List[Set], min_amplitude_heigth: float):
    new_blink_sets = []
    for blink_set in blink_sets:
        values = data[blink_set]
        height = max(values) - min(values)
        if height > min_amplitude_heigth:
            new_blink_sets.append(blink_set)
    return new_blink_sets


MIN_AMPLITUDE_HEIGHT =  std
blink_sets_by_amplitude = filter_blink_sets_by_amplitude(data=data_copy.copy(), blink_sets=blink_sets, min_amplitude_heigth=MIN_AMPLITUDE_HEIGHT)
blink_set_statistics(blink_sets_by_amplitude)

There are 658 blink events with a mean length of 13.09.


In [253]:
slider, ax = plot_blink_sets_start_stop("bo", blink_sets=blink_sets_by_amplitude, data=data, ms=10)
plt.show(block=True)

In [254]:
blink_sets = blink_sets_by_amplitude

### Filter out blink events where first and last value are more than delta away

In [259]:
def filter_blink_sets_by_start_end_delta(data: np.ndarray, blink_sets: List[Set], max_start_end_delta: float):
    new_blink_sets = []
    for blink_set in blink_sets:
        start = data[blink_set[0]]
        end = data[blink_set[-1]]
        if abs(start - end) < max_start_end_delta:
            new_blink_sets.append(blink_set)
    return new_blink_sets

MAX_START_END_DELTA = std
blink_sets = filter_blink_sets_by_start_end_delta(data=data.copy(), blink_sets=blink_sets, max_start_end_delta=MAX_START_END_DELTA)
blink_set_statistics(blink_sets)

There are 520 blink events with a mean length of 13.11.


### Calculate Basic Features

In [347]:
REOPENING_THRESHOLD = .05

class BlinkEvent:
    eye_closure_data = pd.DataFrame(data_copy.copy())
    def __init__(self, indices: List[int]):
        """indices contains the index of each value of one blink event. The first
        marks the start and the last one the end."""
        self.indices = indices
        self.event_data: pd.DataFrame = self.eye_closure_data.iloc[self.indices]
        self._reopen_idx = None
        self._closing_end_idx = None
        self._blink_interval= None
        self._max_closing_speed_idx = None


    @property
    def full_closure_idx(self):
        return self.event_data.idxmax(axis="index")

    @property
    def full_closure(self):
        return self.event_data[self.full_closure_idx]

    @property
    def amplitude(self):
        return (self.event_data.max() - self.event_data.min())#[0]

    @property
    def start_idx(self):
        return self.indices[0]

    @property
    def start(self):
        return self.event_data[self.start_idx]

    @property
    def end_idx(self):
        return self.indices[-1]

    @property
    def end(self):
        return self.event_data[self.end_idx]

    @property
    def reopen_start_idx(self):
        if self._reopen_idx:
            return self._reopen_idx
        reopen_idx = self.full_closure_idx[0]
        for idx in self.indices[self.indices.index(reopen_idx + 1):]:
            if abs(self.event_data.loc[reopen_idx, 0] - self.event_data.loc[idx,0]) < REOPENING_THRESHOLD:
                reopen_idx = idx
            else:
                break
        self._reopen_idx = reopen_idx

        closing_end_idx = self.full_closure_idx[0]
        for idx in self.indices[self.indices.index(reopen_idx - 1)::-1]:
            if abs(self.event_data.loc[closing_end_idx, 0] - self.event_data.loc[idx,0]) < REOPENING_THRESHOLD:
                closing_end_idx = idx
            else:
                break

        self._closing_end_idx = closing_end_idx
        return self._reopen_idx

    @property
    def closing_end_idx(self):
        if self._closing_end_idx:
            return self._closing_end_idx
        _ = self.reopen_start_idx
        return self._closing_end_idx

    @property
    def blink_interval(self):
        if self._blink_interval is None:
            raise ValueError("blink interval can only be set externally.")
        return self._blink_interval

    @property
    def closing_speed(self):
        distance = self.event_data.loc[self.closing_end_idx, 0] - self.event_data.loc[self.start_idx, 0]
        time = self.closing_end_idx - self.start_idx
        return distance/time

    @property
    def max_closing_speed(self):
        closing_data = self.event_data.loc[self.start_idx:self.closing_end_idx, 0]
        z =  closing_data.diff()
        self._max_closing_speed_idx = z[z==z.max()].index[0]
        return z.max()

    @property
    def max_closing_speed_idx(self):
        if self._max_closing_speed_idx:
            return self._max_closing_speed_idx
        _ = self.max_closing_speed_idx
        return self._max_closing_speed_idx

    @property
    def lid_opening_delay(self):
        return self.closing_end_idx - self.reopen_start_idx

blink_events = [BlinkEvent(indices=index_set) for index_set in blink_sets]

In [355]:
def set_blink_intervalls(blink_events: List[BlinkEvent]):
    for i, be in enumerate(blink_events):
        if i == 0:
            be._blink_interval = 0
            continue
        be._blink_interval = be.start_idx - blink_events[i-1].end_idx
    return blink_events

blink_events = set_blink_intervalls(blink_events=blink_events)

In [356]:
for be in blink_events:
    print(be.blink_interval)

0
97
186
62
115
27
129
16
143
276
75
149
722
34
297
238
153
112
519
302
298
161
1640
26
9
1
502
921
129
79
678
356
89
288
3379
0
0
327
233
1003
70
5
62
36
280
96
69
349
48
17
48
227
289
205
38
20
135
157
228
0
32
2
46
52
265
7
259
24
176
69
206
161
65
341
148
58
303
231
388
23
137
212
415
60
280
255
290
243
256
230
962
23
50
94
18
161
19
205
292
2
161
37
36
129
210
156
183
39
623
431
241
414
399
280
393
1603
686
439
318
33
0
81
1055
188
26
95
22
2
60
48
49
71
512
240
325
24
280
722
58
26
37
225
0
22
81
6
282
531
28
247
19
8
40
78
32
0
864
209
114
70
97
2
90
966
281
23
2
11
231
204
40
58
50
89
5
82
229
221
7
8
469
3864
1165
336
380
181
35
122
264
319
229
13
63
41
5
74
29
52
116
161
106
107
24
159
53
17
125
404
25
56
578
706
1127
1701
52
1
92
154
6
904
359
14
52
12
253
44
249
457
431
332
958
187
526
14
358
341
147
82
117
20
44
438
37
1073
219
601
1074
2086
218
55
420
289
748
40
52
1621
168
196
247
26
1253
1370
146
2328
120
353
145
193
169
17
18
370
10
34
124
119
40
2305
49
1073
1169
13
4